In [1]:
#| default_exp updates
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Atualização

> Este módulo atualiza as bases. Executa as queries sql do STEL, RADCOM e baixa os arquivos de estações e plano básico do MOSAICO.

In [2]:
#| export
import os
from decimal import Decimal, getcontext
from typing import Union
import gc

import numpy as np
import pandas as pd
import pyodbc
from rich.console import Console
from rich import print
from pyarrow import ArrowInvalid, ArrowTypeError
from fastcore.xtras import Path
from fastcore.test import test_eq
from fastcore.parallel import parallel
import pyodbc
from pymongo import MongoClient
from dotenv import load_dotenv, find_dotenv

from extracao.icao import get_icao
from extracao.aisgeo import get_aisg
from extracao.aisweb import get_aisw
from extracao.redemet import get_redemet
from extracao.constants import *
from extracao.format import parse_bw, merge_on_frequency, _read_df

getcontext().prec = 5
load_dotenv(find_dotenv())


True

## Conexão com o banco de dados
A função a seguir é um `wrapper` simples que utiliza o `pyodbc` para se conectar ao banco de dados base da Anatel e retorna o objeto da conexão

In [3]:
#| export
def connect_db(
    server: str = "ANATELBDRO05",  # Servidor do Banco de Dados
    database: str = "SITARWEB",  # Nome do Banco de Dados
    trusted_conn: str = "yes",  # Conexão Segura: yes | no
    mult_results: bool = True,  # Múltiplos Resultados
) -> pyodbc.Connection:
    """Conecta ao Banco `server` e retorna o 'cursor' (iterador) do Banco"""
    return pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        f"Server={server};"
        f"Database={database};"
        f"Trusted_Connection={trusted_conn};"
        f"MultipleActiveResultSets={mult_results};",
        timeout=TIMEOUT,
    )


In [4]:
#echo: false
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (SQL_RADCOM,SQL_STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

In [5]:
#| eval: false
test_connection()

In [6]:
#| export
def clean_mosaico(
    df: pd.DataFrame,  # DataFrame com os dados de Estações e Plano_Básico mesclados
    pasta: Union[
        str, Path
    ],  # Pasta com os dados de municípios para imputar coordenadas ausentes
) -> pd.DataFrame:  # DataFrame com os dados mesclados e limpos
    """Clean the merged dataframe with the data from the MOSAICO page"""
    df = df[
        df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$", na=False)
    ].reset_index(drop=True)
    for c in df.columns:
        df.loc[df[c] == "", c] = pd.NA
    df.loc["Frequência"] = df.Frequência.astype("str").str.replace(",", ".")
    df = df[df.Frequência.notna()].reset_index(drop=True)
    df.loc["Frequência"] = df.Frequência.astype("float")
    df.loc[df.Num_Serviço == "205", "Frequência"] = df.loc[
        df.Num_Serviço == "205", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    df.loc[:, "Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
    return df


## Atualização das bases de dados
As bases de dados são atualizadas atráves das funções a seguir, o único argumento passado em todas elas é a pasta na qual os arquivos locais processados serão salvos, os nomes dos arquivos são padronizados e não podem ser editados para que as funções de leitura e processamento recebam somente a pasta na qual esses arquivos foram salvos.

In [7]:
#| export
def _save_df(df: pd.DataFrame, folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Format, Save and return a dataframe"""
    df = df.copy()  # Impedir a alteração do df original
    for c in df.columns:
        df[c] = df[c].astype("string").str.lstrip().str.rstrip()
    df = df.drop_duplicates(keep="first").reset_index(drop=True)
    if "Código_Município" in df:
        df = df[df.Código_Município.notna()].reset_index(drop=True)
    try:
        file = Path(f"{folder}/{stem}.parquet.gzip")
        df.to_parquet(file, compression="gzip", index=False)
    except (ArrowInvalid, ArrowTypeError) as e:
        raise e(f"Não possível salvar o arquivo parquet {file}") from e
    return df


### RADCOM

In [8]:
# | export
def update_radcom(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `RADCOM`, com tratamento de erro de conectividade."""
    console = Console()
    with console.status(
        "[cyan]Lendo o Banco de Dados de Radcom...", spinner="monkey"
    ) as status:
        try:
            return _extract_radcom(conn, folder)
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e


def _extract_radcom(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    df = pd.read_sql_query(SQL_RADCOM, conn)
    df["Entidade"] = df.Entidade.str.rstrip().str.lstrip()
    df["Num_Serviço"] = "231"
    df["Classe_Emissão"] = pd.NA
    df["Largura_Emissão(kHz)"] = "256"
    df["Validade_RF"] = pd.NA
    df["Status"] = "RADCOM"
    df["Fonte"] = "SRD"
    df["Multiplicidade"] = "1"
    a = df.Situação.isna()
    df.loc[a, "Classe"] = df.loc[a, "Fase"]
    df.loc[~a, "Classe"] = (
        df.loc[~a, "Fase"].astype("string")
        + "-"
        + df.loc[~a, "Situação"].astype("string")
    )
    df.drop(["Fase", "Situação"], axis=1, inplace=True)
    df = df.loc[:, COLUNAS]
    return _save_df(df, folder, "radcom")


In [9]:
#| eval: false
import warnings
import os
# warnings.filterwarnings("ignore", message='install "ipywidgets" for Jupyter support')
warnings.filterwarnings("ignore")

In [10]:
%%time
#| eval: false
folder = Path.cwd().parent / 'dados'
conn = connect_db()


CPU times: total: 0 ns
Wall time: 4 ms


In [14]:
%%time
#| eval: false
radcom = update_radcom(conn, folder)
radcom.sample(5)

Output()

CPU times: total: 406 ms
Wall time: 666 ms


,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
3621,104.9,ASSOCIACAO COMUNITARIA CULTURAL ALTO ALEGRE,50012962341,683132571,Cascavel,4104808,PR,-24.945833333333333,-53.4805555555555,3-A,231,<NA>,256,<NA>,RADCOM,SRD,1
890,87.9,ASSOCIAÇÃO DE RADIODIFUSÃO COMUNITÁRIA ESMERALDAS,50407647317,699145384,Esmeraldas,3124104,MG,-19.7580555555555,-44.31388888888883,3,231,<NA>,256,<NA>,RADCOM,SRD,1
1915,87.9,ASSOCIAÇÃO RADIODIFUSÃO COMUNITÁRIA BETEL,50408207809,698144961,Soledade,4320800,RS,-28.826388888888832,-52.506388888888836,3,231,<NA>,256,<NA>,RADCOM,SRD,1
4829,106.3,ASSOCIACAO BENEFICENTE RECANTO CANAA,50416083005,1008215837,São Luís,2111300,MA,-2.5316666666666667,-44.29777777777767,P-A,231,<NA>,256,<NA>,RADCOM,SRD,1
1249,87.9,ACCNR - ASSOCIACAO COMUNIT.CENTRO NORTE DE RAD...,50010479783,598611274,Colíder,5103205,MT,-10.805277777777667,-55.458333333333336,3,231,<NA>,256,<NA>,RADCOM,SRD,1


### STEL

In [12]:
#|export
def update_stel(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `STEL`, com tratamento de erro de conectividade."""
    console = Console()
    with console.status(
        "[red]Lendo o Banco de Dados do STEL",
        spinner="grenade",
    ) as status:
        try:
            return _extract_stel(conn, folder)
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e


def _extract_stel(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `STEL`"""
    stel = pd.read_sql_query(SQL_STEL, conn)
    stel["Status"] = "L"
    stel["Entidade"] = stel.Entidade.str.rstrip().str.lstrip()
    stel["Fonte"] = "STEL"
    stel.loc[:, ["Largura_Emissão(kHz)", "_"]] = (
        stel.Largura_Emissão.fillna("").apply(parse_bw).tolist()
    )
    stel.drop(["Largura_Emissão", "_"], axis=1, inplace=True)
    stel.loc[:, "Validade_RF"] = stel.Validade_RF.astype("string").str.slice(0, 10)
    stel.loc[stel.Unidade == "kHz", "Frequência"] = stel.loc[
        stel.Unidade == "kHz", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    stel.loc[stel.Unidade == "GHz", "Frequência"] = stel.loc[
        stel.Unidade == "GHz", "Frequência"
    ].apply(lambda x: Decimal(x) * Decimal(1000))
    stel.drop("Unidade", axis=1, inplace=True)
    stel["Multiplicidade"] = 1
    stel = stel.loc[:, COLUNAS]
    return _save_df(stel, folder, "stel")


In [15]:
%%time
#| eval: false
stel = update_stel(conn, folder)
stel.sample(5)

Output()

CPU times: total: 3.98 s
Wall time: 29.6 s


,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
23384,767.0,TELEVISAO RIO FORMOSO LTDA,13030154475,3298213,Miranorte,1713304,TO,-9.535833333333333,-48.576111111111,TX,728,C3F,6000.0,1997-10-20,L,STEL,1
14150,156.55,VOPAK BRASIL S.A.,50415542502,1006008893,Santos,3548500,SP,-23.926388888888834,-46.372777777777664,FC,604,F3E,11.0,2037-10-07,L,STEL,1
31795,25.375,EDIFICIO DOUBLE SPACE FARIA LIMA,50403887305,688907423,São Paulo,3550308,SP,-23.591116666666668,-46.68056944444433,TX,060,F3E,16.0,2016-10-24,L,STEL,1
2056,131.875,AZUL LINHAS AEREAS BRASILEIRAS S.A,50405707169,1009310787,São Gonçalo do Amarante,2412005,RN,-5.763611111111,-35.37222222222216,FA,507,A3E,6.0,2029-01-07,L,STEL,1
20432,156.75,PETROLEO BRASILEIRO S A PETROBRAS,50411168908,700046925,Santos,3548500,SP,-25.266938888888834,-45.252811111111,FC,604,G3E,16.0,2033-12-03,L,STEL,1


### MOSAICO - SRD

In [16]:
#|export
def update_srd(
    mongo_client: MongoClient,  # Objeto de conexão com o MongoDB
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Efetua a query na tabela de Radiodifusão no banco mongoDB `mongo_client` e atualiza o arquivo local"""
    console = Console()
    with console.status(
        "Consolidando os dados do Mosaico...", spinner="runner"
    ) as status:
        database = mongo_client["sms"]
        collection = database["srd"]
        list_data = list(collection.find(MONGO_SRD, projection=COLS_SRD.keys()))
        mosaico = pd.json_normalize(list_data)
        mosaico = mosaico.drop(columns=["estacao"])
        mosaico = mosaico[list(COLS_SRD.keys())]
        mosaico.rename(COLS_SRD, axis=1, inplace=True)
        mosaico = clean_mosaico(mosaico, folder)
        mosaico["Fonte"] = "MOS"
        mosaico["Num_Serviço"].fillna("", inplace=True)
        mosaico.loc[:, ["Largura_Emissão(kHz)", "Classe_Emissão"]] = (
            mosaico.Num_Serviço.astype("string")
            .fillna("")
            .map(BW_MAP)
            .apply(parse_bw)
            .tolist()
        )
        mosaico.loc[mosaico.Classe_Emissão == "", "Classe_Emissão"] = pd.NA
        mosaico["Multiplicidade"] = 1
        mosaico = mosaico.loc[:, COLUNAS]
    return _save_df(mosaico, folder, "srd")


In [17]:
#|eval: false
uri = os.environ['MONGO_URI']
mongo_client = MongoClient(uri)
mongo_client.server_info()

{'version': '4.0.5',
 'gitVersion': '3739429dd92b92d1b0ab120911a23d50bf03c412',
 'targetMinOS': 'Windows 7/Windows Server 2008 R2',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 0, 5, 0],
 'openssl': {'running': 'Windows SChannel'},
 'buildEnvironment': {'distmod': '2008plus-ssl',
  'distarch': 'x86_64',
  'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'ccflags': '/nologo /EHsc /W3 /wd4355 /wd4800 /wd4267 /wd4244 /wd4290 /wd4068 /wd4351 /wd4373 /we4013 /we4099 /we4930 /WX /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /Zc:rvalueCast /Zc:strictStrings /volatile:iso /Gw /Gy /Zc:inline',
  'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'cxxflags': '/TP',
  'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF',
  'target_arch': 'x86_64',
  'target_os': 'windows'},
 'bits': 64,
 'debug': False,
 'maxBsonObjectSize': 16777216,
 '

In [18]:
%%time
#|eval: false
mosaico = update_srd(mongo_client, folder)
mosaico.sample(5)

Output()

CPU times: total: 3 s
Wall time: 3.75 s


,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
6112,57.0,PREFEITURA MUNICIPAL DE SAO BONIFACIO,14023501395,323090044,São Bonifácio,4215901,SC,-27.9013888888888333,-48.9291666666666666,C,800,<NA>,6000.0,2018-12-31,TV-C7,MOS,1
10493,533.0,TELEVISAO INDEPENDENTE DE SAO JOSE DO RIO PRET...,50409648620,699602360,Cajazeiras,2503704,PB,-6.8947277777776666,-38.5508333333333333,A,801,<NA>,5700.0,2027-07-30,TV-C4,MOS,1
3721,213.0,PREFEITURA MUNICIPAL DE SONORA,50400553821,322828732,Sonora,5007935,MS,-17.5802777777776666,-54.7505555555555000,C,800,<NA>,6000.0,2018-12-31,TV-C7,MOS,1
16865,107.3,BRASIL AMAZONIA COMUNICACAO E EMPREENDIMENTOS ...,50001791109,323688144,Nova Timboteua,1505007,PA,-1.1997222222221666,-47.3977777777776666,A3,230,<NA>,256.0,2028-12-08,FM-C4,MOS,1
11089,521.0,TELEVISAO ANHANGUERA DE ARAGUAINA LTDA,50411145363,1005698659,Augustinópolis,1702554,TO,<NA>,<NA>,C,801,<NA>,5700.0,2028-10-03,TV-C3,MOS,1


### MOSAICO - TELECOM

In [19]:
#| export
def update_telecom(
    mongo_client: MongoClient,  # Objeto de conexão com o MongoDB
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Efetua a query na tabela `licenciamento` no banco mongoDB `mongo_client` e atualiza o arquivo local"""

    database = mongo_client["sms"]
    collection = database["licenciamento"]
    query = collection.find(
        MONGO_TELECOM, projection={k: 1.0 for k in COLS_TELECOM.keys()}, limit=0
    )
    print(
        "[red] :warning: Executando a query na base licenciamento do Mosaico, processo demorado! :warning:"
    )
    df = pd.DataFrame(list(query), columns=COLS_TELECOM.keys(), dtype="string")
    path_cache = Path(f"{folder}/telecom_raw.parquet.gzip")
    path_out = Path(f"{folder}/telecom.parquet.gzip")
    if path_cache.is_file():
        cache_df = pd.read_parquet(path_cache)
        if df.equals(cache_df) and path_out.is_file():
            del df
            gc.collect()
            return pd.read_parquet(path_out)
    df.to_parquet(path_cache, compression="gzip", index=False)
    return _process_telecom(df, folder)


def _process_telecom(
    df: pd.DataFrame,  # Dataframe não processado de dados do Mosaico
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:
    """Formata e pós-processa e mescla os dados de Telecomunicações do Mosaico"""
    # df.drop("_id", axis=1, inplace=True)
    df.rename(COLS_TELECOM, axis=1, inplace=True)
    df["Designacao_Emissão"] = df.Designacao_Emissão.str.replace(",", " ")
    df["Designacao_Emissão"] = (
        df.Designacao_Emissão.str.strip().str.lstrip().str.rstrip().str.upper()
    )
    df["Designacao_Emissão"] = df.Designacao_Emissão.str.split(" ")
    df = df.explode("Designacao_Emissão", ignore_index=True)
    df.loc[df.Designacao_Emissão == "/", "Designacao_Emissão"] = ""
    df.loc[:, ["Largura_Emissão(kHz)", "Classe_Emissão"]] = df.Designacao_Emissão.apply(
        parse_bw
    ).tolist()
    df.drop("Designacao_Emissão", axis=1, inplace=True)
    subset = [
        "Frequência",
        "Entidade",
        "Fistel",
        "Código_Município",
        "Longitude",
        "Latitude",
        "Classe",
        "Num_Serviço",
        "Classe_Emissão",
        "Largura_Emissão(kHz)",
    ]
    df.dropna(subset=subset, axis=0, inplace=True)
    df_sub = (
        df[~df.duplicated(subset=subset, keep="first")].reset_index(drop=True).copy()
    )
    df_sub["Multiplicidade"] = (
        df.groupby(subset, sort=False).count()["Número_Estação"]
    ).tolist()
    df_sub["Status"] = "L"
    df_sub["Fonte"] = "MOS"
    del df
    gc.collect()
    df_sub = df_sub.reset_index()
    df_sub = df_sub.loc[:, COLUNAS]
    return _save_df(df_sub, folder, "telecom")


In [20]:
%%time
#| eval: false
telecom = update_telecom(mongo_client, folder)
telecom.sample(5)
# telecom = pd.read_parquet(folder / 'telecom.parquet.gzip')

 ⚠ Executando a query na base licenciamento do Mosaico, processo demorado! ⚠

CPU times: total: 7min
Wall time: 17min 11s


,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
63467,159.01875,VALLOUREC SOLUÇÕES TUBULARES DO BRASIL S.A.,50417301219,1000873517,,3106200,MG,-19.963417,-44.009833,ML,019,F1E,8.0,2035-05-07,L,MOS,300
413564,17920.0,CLARO S.A.,50418766738,1013144284,PAULÍNIA,3536505,SP,-22.77703,-47.13537,FX,019,D7W,27500.0,2039-11-28,L,MOS,1
740175,874.5,CLARO S.A.,11021017965,1014405804,MANAUS,1302603,AM,-3.10803,-59.9917,FB,175,G9W,5000.0,2023-04-30,L,MOS,3
465027,23418.5,TELEFONICA BRASIL S.A.,50417179405,1014985797,PARAOPEBA,3147402,MG,-19.286192,-44.404917,FX,019,G7W,7000.0,2039-02-08,L,MOS,1
747498,1875.0,CLARO S.A.,11021017965,1014394420,VOLTA REDONDA,3306305,RJ,-22.52665,-44.122277,FB,175,D7W,5000.0,2027-07-02,L,MOS,3


### AERONAUTICA

In [21]:
#| export 
def update_aero(
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a base de dados de emissões da aeronáutica"""
    icao = get_icao()
    aisw = get_aisw()
    aisg = get_aisg()
    redemet = get_redemet()
    radares = pd.read_excel(os.environ["PATH_RADAR"])
    for df in [aisw, aisg, redemet, radares]:
        icao = merge_on_frequency(icao, df)
    icao = icao.astype({"Frequency": 'float64', 
                 "Latitude": 'float32',
                 "Longitude": 'float32',
                 "Description": 'string'})
    # TODO: Eliminate this eventually
    icao.loc[np.isclose(icao.Longitude, -472.033447), "Longitude"] = -47.2033447
    icao.loc[np.isclose(icao.Longitude, 69.934998), "Longitude"] = -69.934998
    return _save_df(icao, folder, "aero")


In [22]:
%%time
#| eval: false
aero = update_aero(folder)
aero.sample(5)

CPU times: total: 6min 51s
Wall time: 1min 58s


,Frequency,Latitude,Longitude,Description
132,121.25,-16.140556,-48.57694625854492,"[ICAO] APP-U C-150/450, ANÁPOLIS"
3335,117.5,-15.865013,-47.90018844604492,[AISG] VOR - KUBITSCHEK CH 122X
1265,131.3,-3.041111,-60.05055618286133,"[ICAO] AOC U 100/100, MANAUS"
1024,130.275,-23.435556,-46.47305679321289,"[ICAO] AOC U 100/100, GUARULHOS"
446,126.05,-15.84,-52.27777862548828,"[ICAO] ACC-U C-261/450, BARRA DO GARCAS"


In [23]:
#| export
def validar_coords(
    row: pd.Series,  # Linha de um DataFrame
    connector: pyodbc.Connection = None,  # Conector de Banco de Dados
) -> tuple:  # DataFrame com dados do município
    """Valida os dados de coordenadas e município em `row` no polígono dos municípios em banco corporativ do IBGE"""

    mun, cod, lat, long = (
        row.Município,
        row.Código_Município,
        row.Latitude,
        row.Longitude,
    )
    is_valid = "-1"
    conn = connect_db() if connector is None else connector
    crsr = conn.cursor()
    sql = SQL_VALIDA_COORD.format(long, lat, cod)
    crsr.execute(sql)
    result = crsr.fetchone()
    if result is not None:
        mun = result.NO_MUNICIPIO
        lat = result.NU_LATITUDE
        long = result.NU_LONGITUDE
        is_valid = result.COORD_VALIDA
    if connector is None:
        del conn
    return [str(mun), str(lat), str(long), str(is_valid)]


In [24]:
#| export
def update_cached_df(df: pd.DataFrame, df_cache: pd.DataFrame) -> pd.DataFrame:
    """Mescla ambos dataframes eliminando os excluídos (existentes somente em df_cache)"""

    # Merge dataframes based on all columns except "Coords_Valida_IBGE"
    merged = pd.merge(
        df_cache,
        df,
        on=list(df.columns),
        how="outer",
        indicator=True,
        copy=False,
        validate="one_to_one",
    )

    # Identify rows only present in df_cache
    # df_cache_only = merged[merged["_merge"] == "left_only"] #TODO: Data logging

    # Identify news rows
    # df_new = merged[merged["_merge"] == "right_only"] #TODO: Data logging

    # Exclude rows only present in df_cache
    df_cache = merged[merged["_merge"] != "left_only"]

    # inplace=True not working
    df_cache.loc[:, ["Latitude", "Longitude"]] = df_cache.loc[
        :, ["Latitude", "Longitude"]
    ].fillna("-1")

    # # Drop the _merge column
    return df_cache.drop(columns="_merge")


In [25]:
# | export
def _validar_coords_base(
    df: pd.DataFrame,  # DataFrame com os dados da Anatel
    df_cache: pd.DataFrame,  # DataFrame validado anteriormente, usado como cache
) -> pd.DataFrame:  # DataFrame com as coordenadas validadas na base do IBGE
    """Valida as coordenadas consultado a Base Corporativa do IBGE, excluindo o que já está no cache na versão anterior"""

    ibge = ["Município_IBGE", "Latitude_IBGE", "Longitude_IBGE", "Coords_Valida_IBGE"]

    df_cache = update_cached_df(df, df_cache)

    subset = df_cache.Coords_Valida_IBGE.isna()

    linhas = list(
        df_cache.loc[
            subset, ["Município", "Código_Município", "Latitude", "Longitude"]
        ].itertuples()
    )

    df_cache.loc[subset, ibge] = parallel(
        validar_coords, linhas, threadpool=True, n_workers=20, progress=True
    )

    df_cache.loc[df_cache.Coords_Valida_IBGE == "-1", "Coords_Valida_IBGE"] = pd.NA

    return df_cache


In [26]:
# | export
def update_base(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    clientMongoDB: MongoClient,  # Objeto de conexão com o MongoDB
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    # sourcery skip: use-fstring-for-concatenation
    """Wrapper que atualiza opcionalmente lê e atualiza as 4 bases indicadas anteriormente, as combina e salva o arquivo consolidado na folder `folder`"""
    stel = update_stel(conn, folder)
    radcom = update_radcom(conn, folder)
    mosaico = update_srd(clientMongoDB, folder)
    telecom = update_telecom(clientMongoDB, folder)

    df = (
        pd.concat([mosaico, radcom, stel, telecom])
        .sort_values(["Frequência", "Latitude", "Longitude"])
        .reset_index(drop=True)
    )

    for c in df.columns:
        df[c] = df[c].astype("string")

    # inplace not working!
    df.loc[:, ["Latitude", "Longitude"]] = df.loc[:, ["Latitude", "Longitude"]].fillna(
        "0"
    )

    df_cache = _read_df(folder, "base")

    df_cache = _validar_coords_base(df, df_cache)

    return _save_df(df_cache, folder, "base")

In [ ]:
#| eval: false
# base = update_base(conn, mongo_client, folder)
# base.sample(5)

Output()

Output()

Output()

 ⚠ Executando a query na base licenciamento do Mosaico, processo demorado! ⚠